In [1]:
from scipy.stats import multivariate_normal
import numpy as np

In [3]:
## set up multivariate norm used throughout tests
# 40% from mean = [2, 5], cov = [[1, .5], [.5, 100]]
# 60% from mean = [6, 9], cov = [[5, 3], [3, 40]]

mvn1 = multivariate_normal(mean = [2, 5], cov = [[1, .5], [.5, 100]])
mvn2 = multivariate_normal(mean = [6, 9], cov = [[5, 3], [3, 40]])
x11, x12 = 2.449356520423142, 22.228603905059355
pdfx1 = mvn1.pdf([x11, x12])
pdfx2 = mvn2.pdf([x11, x12])
pdfx1_weighted = pdfx1 * 0.4
pdfx2_weighted = pdfx2 * 0.6
print(pdfx1, pdfx2)
print(pdfx1_weighted, pdfx2_weighted)

## more data
x_vals = [[2.449356520423142, 22.228603905059355],
            [7.375567311048346, 7.658930529870852],
            [0.6206099136380241, -0.5206737056553434],
            [4.023896000998747, -1.1727896397005733],
            [7.794096850083458, 7.068473377153386]]

## weigth values (comes up in M step)
wk_vals = [[0.98630233, 0.01369767],
                [0.93771556, 0.06228444],
                [0.62419835, 0.37580165],
                [0.98378211, 0.01621789],
                [0.9500608, 0.0499392]]

0.0033814120216 0.000148875770138
0.00135256480864 8.9325462083e-05


In [4]:
## eStepDatumTest
denom = pdfx1_weighted + pdfx2_weighted
print(pdfx1_weighted/denom, pdfx2_weighted/denom)

0.9380497504583296 0.06195024954167031


In [23]:
## logLikelihoodGMMTest
np.log(pdfx1_weighted + pdfx2_weighted)

-6.541800338213515

In [6]:
list(np.random.uniform(size=5))

[0.19765004055219848,
 0.36498221099062,
 0.1968909878188082,
 0.5009962966680894,
 0.7966995493558133]

In [6]:
wk_vals = list(map(np.array,wk_vals))
wk_vals = np.array(wk_vals)
print(type(wk_vals), type(wk_vals[0]))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [7]:
wk_vals

array([[ 0.98630233,  0.01369767],
       [ 0.93771556,  0.06228444],
       [ 0.62419835,  0.37580165],
       [ 0.98378211,  0.01621789],
       [ 0.9500608 ,  0.0499392 ]])

In [5]:
## functions for mean m step
def generate_wk(n):
    l1 = list(np.random.uniform(size = n))
    l2 = [1 - x for x in l1]
    return [np.array([x1,x2]) for x1,x2 in zip(l1,l2)]


def mean_calc(x_vals, wklist, nklist, kposition=0):
    #look up vector class for sum_val?
    k = len(nklist)
    n = len(x_vals)
    sum_val = np.array([0] * k)
    for i in range(n):
        sum_val = np.add(sum_val, np.multiply(wklist[i][kposition], np.array(x_vals[i])))
        
    return sum_val/nklist[kposition]

In [77]:
wk_weights = wk_vals
nk_weights = [1.4, 3.6]
mean1 = mean_calc(x_vals, wk_weights, nk_weights)
print(mean1)
print("*******")
mean2 = mean_calc(x_vals, wk_weights, nk_weights,1)
print(mean2)
means = [mean1, mean2]

[ 15.05919768  24.53052035]
*******
[ 0.32795829  0.25550444]


In [35]:
x_vals

[[2.449356520423142, 22.228603905059355],
 [7.375567311048346, 7.658930529870852],
 [0.6206099136380241, -0.5206737056553434],
 [4.023896000998747, -1.1727896397005733],
 [7.794096850083458, 7.068473377153386]]

In [46]:
## covariance matrix M step end. Result should be collection of covariance matrices
x_vals_matrix = [np.array([xi]) for xi in x_vals]
print(x_vals_matrix)

[array([[  2.44935652,  22.22860391]]), array([[ 7.37556731,  7.65893053]]), array([[ 0.62060991, -0.52067371]]), array([[ 4.023896  , -1.17278964]]), array([[ 7.79409685,  7.06847338]])]


In [56]:
print(x_vals_matrix[0], mean1, x_vals_matrix[0] - mean1, (x_vals_matrix[0] - mean1).transpose())

[[  2.44935652  22.22860391]] [ 15.05919768  24.53052035] [[-12.60984115  -2.30191645]] [[-12.60984115]
 [ -2.30191645]]


In [97]:
def cov_calc(x_vals, wklist, meanlist, nklist, kposition=0):
    k = len(nklist)
    n = len(x_vals)
    d = len(x_vals[0])
    sumtotal = np.array([[0] * d] * d)
    for i in range(n):
        x_minus_mean = np.array([x_vals[i] - meanlist[kposition]])
        sumval = wklist[i][kposition] * np.multiply(x_minus_mean, x_minus_mean.transpose())
        sumtotal = np.add(sumtotal, sumval)
    return (1/nklist[kposition]) * sumtotal

In [98]:
cov1 = cov_calc(x_vals, wk_vals, means, nk_weights)
print(cov1)
print('*****')
cov2 = cov_calc(x_vals, wk_vals, means, nk_weights, kposition=1)
print(cov2)

[[  512.2681749    775.53671376]
 [  775.53671376  1603.51194885]]
*****
[[  6.19273023   6.25754029]
 [  6.25754029  12.60481687]]
